In [3]:
import pandas as pd
import regex as re
import numpy as np

In [4]:
groups_22 = pd.read_csv('2022_world_cup_groups.csv')

In [5]:
matches_22 = pd.read_csv('2022_world_cup_matches.csv')

In [6]:
data_dict = pd.read_csv('data_dictionary.csv') ## -> fields description

In [188]:
international_matches = pd.read_csv('international_matches.csv')
international_matches.drop(columns={'ID'}, inplace= True)
international_matches.isnull().sum() #Win conditions have null values when matches have ended normally
international_matches['Win Conditions'] = international_matches['Win Conditions'].fillna('normal ending')

In [189]:
international_matches.rename(columns={"Tournament":"Stage","Home Stadium":"Host Team"},inplace = True)

In [190]:
col_list = list(international_matches)

In [191]:
international_matches = international_matches.reindex(columns=['Date','Stage','Home Team','Home Goals',
                                                               'Away Goals','Away Team','Win Conditions','Host Team'])

In [192]:
world_cup_matches = pd.read_csv('world_cup_matches.csv')
world_cup_matches.drop(columns={'ID'}, inplace= True)
world_cup_matches.isnull().sum() #Win conditions have null values when matches have ended normally
world_cup_matches['Win Conditions'] = world_cup_matches['Win Conditions'].fillna('normal ending')

In [193]:
world_cup_matches.drop(columns={'Year'},inplace=True)

In [194]:
history_matches = pd.concat([international_matches, world_cup_matches], ignore_index=True)

In [195]:
history_matches.head(5)

,Date,Stage,Home Team,Home Goals,Away Goals,Away Team,Win Conditions,Host Team
0,1872-11-30,Friendly,Scotland,0,0,England,normal ending,True
1,1873-03-08,Friendly,England,4,2,Scotland,normal ending,True
2,1874-03-07,Friendly,Scotland,2,1,England,normal ending,True
3,1875-03-06,Friendly,England,2,2,Scotland,normal ending,True
4,1876-03-04,Friendly,Scotland,3,0,England,normal ending,True


## Football History of Spain

In [53]:
spain_matches = international_matches[(history_matches['Home Team']=='Spain') | (history_matches['Away Team']=='Spain')]

C:\Users\juanp\AppData\Local\Temp\ipykernel_18504\1685220102.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  spain_matches = international_matches[(history_matches['Home Team']=='Spain') | (history_matches['Away Team']=='Spain')]


In [162]:
def win_conditions(row):
    if 'penalties' in row['Win Conditions']:
        return 'penalties'
    else:
        return row['Win Conditions']

In [140]:
def home_winner(row):
    # This functions will return if Spain either won or lost a match 
    if row['Home Goals'] > row['Away Goals']:
        return 'Win'
    elif row['Home Goals'] == row['Away Goals'] and row['Win Conditions'] == 'normal ending':
        return 'Draw'
    elif row['Home Goals'] == row['Away Goals'] and row['Home Team'] in row['Win Conditions']:
        return 'Win'
    else:
        return 'Loss'

In [166]:
def away_winner(row):
    if row['Away Goals'] > row['Home Goals']:
        return 'Win'
    elif row['Home Goals'] == row['Away Goals'] and row['Win Conditions'] == 'normal ending':
        return 'Draw'
    elif row['Home Goals'] == row['Away Goals'] and row['Away Team'] in row['Win Conditions']:
        return 'Win'
    else:
        return 'Loss'

In [211]:
def setting_history_matches(df):
    df['win_conditions'] =  df.apply(lambda row: win_conditions(row), axis=1)
    df['home_winner'] = df.apply(lambda row: home_winner(row), axis=1)
    df['away_winner'] = df.apply(lambda row: away_winner(row), axis=1)
    
    df = df.drop(columns={'Win Conditions'})
    
    return df

In [212]:
final_df = setting_history_matches(history_matches)

In [269]:
final_df['match'] = final_df['Home Team']+ ' vs ' + final_df['Away Team']

In [277]:
home = final_df[final_df.columns.difference(['Away Team', 'Away Goals', 'away_winner'])]

In [278]:
away = final_df[final_df.columns.difference(['Home Team', 'Home Goals', 'home_winner'])]

In [279]:
home = home.loc[:, ['Date', 'Stage', 'Home Team', 'Home Goals', 'Host Team', 'home_winner', 'win_conditions', 'match']]
home.rename(columns={'Home Team':'Team', 'Home Goals':'Goals', 'home_winner':'Result'}, inplace=True)

In [280]:
away = away.loc[:, ['Date', 'Stage', 'Away Team', 'Away Goals', 'Host Team', 'away_winner', 'win_conditions', 'match']]
away.rename(columns={'Away Team':'Team', 'Away Goals':'Goals', 'away_winner':'Result'}, inplace=True)

In [282]:
df = pd.concat([home, away])

In [334]:
merged_df = pd.merge(df, df, left_on=['Date','match'], right_on=['Date','match'])

In [335]:
all_results = test.drop_duplicates(subset=['Date','match'],keep='last')

In [336]:
all_results = test[~(test['Team_x'] == test['Team_y'])]

In [342]:
final_result = all_results.loc[:,['Date', 'Stage_x', 'Team_x', 'Goals_x', 
                                  'Team_y', 'Goals_y','Result_x','win_conditions_x','Host Team_x']]

In [343]:
final_result.rename(columns={'Stage_x':'Stage','Team_x':'Team','Goals_x':'Home Goals',
                             'Team_y':'Team Against','Goals_y':'Against Goals', 'Result_x':'Result', 
                             'win_conditions_x':'Win Conditions','Host Team_x':'Host Team'},inplace=True)

In [345]:
final_result.to_csv('C:\\Users\\juanp\\proyectos_varios\\world_cup\\history_matches.csv')